# 📄 Question Paper Processing Pipeline

This notebook extracts structured data from question papers:

1. **Paper Metadata** - Title, subject, total marks, time limit
2. **Question Extraction** - Question text, marks, number
3. **Section Detection** - Group questions by section
4. **Answer Key Association** - Link expected answers

---

## 1. Setup & Imports

In [ ]:
# Install dependencies if needed
# !pip install pypdf2 pdf2image pytesseract pillow opencv-python

In [ ]:
import re
import json
from pathlib import Path
from typing import List, Dict, Optional, Tuple
from dataclasses import dataclass, field, asdict
import warnings
warnings.filterwarnings('ignore')

# PDF handling
try:
    from PyPDF2 import PdfReader
    PDF_AVAILABLE = True
except ImportError:
    PDF_AVAILABLE = False
    print("⚠️ PyPDF2 not installed. PDF support disabled.")

# OCR for scanned papers
try:
    import pytesseract
    from PIL import Image
    OCR_AVAILABLE = True
except ImportError:
    OCR_AVAILABLE = False
    print("⚠️ pytesseract not installed. OCR support disabled.")

print(f"PDF support: {PDF_AVAILABLE}")
print(f"OCR support: {OCR_AVAILABLE}")

---
## 2. Data Structures

In [ ]:
@dataclass
class Question:
    """Represents a single question from a paper."""
    number: str  # e.g., "1", "2a", "3.i"
    text: str
    marks: float
    section: Optional[str] = None  # e.g., "Section A", "Part I"
    question_type: str = "short_answer"  # short_answer, long_answer, mcq, fill_blank
    expected_answer: Optional[str] = None
    keywords: List[str] = field(default_factory=list)
    sub_questions: List['Question'] = field(default_factory=list)


@dataclass
class QuestionPaper:
    """Complete question paper structure."""
    title: str
    subject: str
    total_marks: float
    time_limit_minutes: int
    instructions: List[str] = field(default_factory=list)
    sections: List[str] = field(default_factory=list)
    questions: List[Question] = field(default_factory=list)
    raw_text: str = ""
    
    def to_dict(self) -> dict:
        """Convert to dictionary for JSON serialization."""
        return {
            "title": self.title,
            "subject": self.subject,
            "total_marks": self.total_marks,
            "time_limit_minutes": self.time_limit_minutes,
            "instructions": self.instructions,
            "sections": self.sections,
            "questions": [asdict(q) for q in self.questions],
            "question_count": len(self.questions)
        }


# Example
sample_question = Question(
    number="1",
    text="State Newton's first law of motion.",
    marks=2.0,
    section="Section A",
    question_type="short_answer",
    keywords=["rest", "motion", "external force", "inertia"]
)
print(f"Sample question: {sample_question.number} ({sample_question.marks} marks)")

---
## 3. Text Extraction (PDF & Image)

In [ ]:
def extract_text_from_pdf(pdf_path: str) -> str:
    """
    Extract text from a PDF file.
    """
    if not PDF_AVAILABLE:
        raise ImportError("PyPDF2 is required for PDF processing")
    
    reader = PdfReader(pdf_path)
    text_parts = []
    
    for page in reader.pages:
        text = page.extract_text()
        if text:
            text_parts.append(text)
    
    return "\n".join(text_parts)


def extract_text_from_image(image_path: str) -> str:
    """
    Extract text from an image using OCR.
    """
    if not OCR_AVAILABLE:
        raise ImportError("pytesseract is required for OCR")
    
    image = Image.open(image_path)
    text = pytesseract.image_to_string(image)
    return text


def extract_text(file_path: str) -> str:
    """
    Extract text from any supported file type.
    """
    path = Path(file_path)
    suffix = path.suffix.lower()
    
    if suffix == '.pdf':
        return extract_text_from_pdf(file_path)
    elif suffix in ['.png', '.jpg', '.jpeg', '.webp', '.tiff']:
        return extract_text_from_image(file_path)
    elif suffix == '.txt':
        return path.read_text()
    else:
        raise ValueError(f"Unsupported file type: {suffix}")


print("✅ Text extraction functions defined")

---
## 4. Paper Metadata Extraction

In [ ]:
class MetadataExtractor:
    """
    Extract paper metadata: title, subject, marks, time.
    """
    
    # Regex patterns for common formats
    PATTERNS = {
        'total_marks': [
            r'total\s*(?:marks?)?\s*[:\-]?\s*(\d+)',
            r'maximum\s*marks?\s*[:\-]?\s*(\d+)',
            r'max\.?\s*marks?\s*[:\-]?\s*(\d+)',
            r'(\d+)\s*marks?\s*(?:total|maximum)',
            r'marks?\s*[:\-]\s*(\d+)',
        ],
        'time_limit': [
            r'time\s*(?:allowed)?\s*[:\-]?\s*(\d+)\s*(?:hours?|hrs?)',
            r'time\s*[:\-]?\s*(\d+)\s*(?:minutes?|mins?)',
            r'duration\s*[:\-]?\s*(\d+)\s*(?:hours?|hrs?)',
            r'(\d+)\s*(?:hours?|hrs?)\s*(?:allowed)?',
        ],
        'subject': [
            r'subject\s*[:\-]?\s*([A-Za-z\s]+)',
            r'(?:physics|chemistry|biology|mathematics|english|history|geography)',
        ]
    }
    
    SUBJECTS = ['physics', 'chemistry', 'biology', 'mathematics', 'math', 
                'english', 'history', 'geography', 'science', 'economics']
    
    @classmethod
    def extract_total_marks(cls, text: str) -> float:
        """Extract total marks from paper text."""
        text_lower = text.lower()
        
        for pattern in cls.PATTERNS['total_marks']:
            match = re.search(pattern, text_lower)
            if match:
                return float(match.group(1))
        
        return 0.0
    
    @classmethod
    def extract_time_limit(cls, text: str) -> int:
        """Extract time limit in minutes."""
        text_lower = text.lower()
        
        for pattern in cls.PATTERNS['time_limit']:
            match = re.search(pattern, text_lower)
            if match:
                value = int(match.group(1))
                # Convert hours to minutes if needed
                if 'hour' in pattern or 'hr' in pattern:
                    return value * 60
                return value
        
        return 0
    
    @classmethod
    def extract_subject(cls, text: str) -> str:
        """Detect subject from text."""
        text_lower = text.lower()
        
        for subject in cls.SUBJECTS:
            if subject in text_lower:
                return subject.capitalize()
        
        return "General"
    
    @classmethod
    def extract_title(cls, text: str) -> str:
        """Extract paper title from first few lines."""
        lines = text.strip().split('\n')[:10]
        
        for line in lines:
            line = line.strip()
            # Look for exam-like titles
            if any(keyword in line.lower() for keyword in 
                   ['exam', 'test', 'assessment', 'paper', 'quiz', 'midterm', 'final']):
                return line[:100]  # Limit title length
            # Or use first substantial line
            if len(line) > 10 and not line.startswith(('1', '2', 'Q', 'q')):
                return line[:100]
        
        return "Question Paper"


# Test
sample_text = """
PHYSICS EXAMINATION - Class XII
Maximum Marks: 70
Time Allowed: 3 Hours

Instructions:
1. All questions are compulsory
2. Section A has 20 questions of 1 mark each

Section A
1. State Newton's first law of motion. (2 marks)
"""

print(f"Title: {MetadataExtractor.extract_title(sample_text)}")
print(f"Subject: {MetadataExtractor.extract_subject(sample_text)}")
print(f"Total Marks: {MetadataExtractor.extract_total_marks(sample_text)}")
print(f"Time: {MetadataExtractor.extract_time_limit(sample_text)} minutes")

---
## 5. Question Extraction

In [ ]:
class QuestionExtractor:
    """
    Extract individual questions from paper text.
    """
    
    # Patterns to identify question starts
    QUESTION_PATTERNS = [
        # "1. Question text" or "1) Question text"
        r'^\s*(\d+)[.)]\s+(.+?)(?=\n\s*\d+[.)]|\n\s*(?:Section|SECTION)|$)',
        # "Q1. Question text" or "Q.1 Question text"
        r'^\s*[Qq]\.?\s*(\d+)[.)]?\s+(.+?)(?=\n\s*[Qq]|\n\s*(?:Section|SECTION)|$)',
        # "(a) Sub question"
        r'^\s*\(([a-z])\)\s+(.+?)(?=\n\s*\([a-z]\)|$)',
    ]
    
    # Patterns for marks
    MARK_PATTERNS = [
        r'\((\d+(?:\.\d+)?)\s*(?:marks?|m)\)',  # (2 marks) or (2m)
        r'\[(\d+(?:\.\d+)?)\s*(?:marks?|m)?\]',  # [2] or [2 marks]
        r'(\d+(?:\.\d+)?)\s*marks?$',  # 2 marks at end
    ]
    
    # Section patterns
    SECTION_PATTERNS = [
        r'Section\s+([A-Z])',
        r'SECTION\s+([A-Z])',
        r'Part\s+([IVX]+|\d+)',
    ]
    
    @classmethod
    def extract_marks(cls, text: str) -> Tuple[str, float]:
        """
        Extract marks from question text.
        Returns (cleaned_text, marks)
        """
        for pattern in cls.MARK_PATTERNS:
            match = re.search(pattern, text, re.IGNORECASE)
            if match:
                marks = float(match.group(1))
                cleaned = re.sub(pattern, '', text, flags=re.IGNORECASE).strip()
                return cleaned, marks
        
        return text.strip(), 0.0
    
    @classmethod
    def detect_question_type(cls, text: str) -> str:
        """Detect type of question."""
        text_lower = text.lower()
        
        if any(kw in text_lower for kw in ['choose', 'select', 'option', 'a)', 'b)', 'c)', 'd)']):
            return 'mcq'
        elif any(kw in text_lower for kw in ['fill', 'blank', '____', '___']):
            return 'fill_blank'
        elif any(kw in text_lower for kw in ['true', 'false', 't/f']):
            return 'true_false'
        elif any(kw in text_lower for kw in ['explain', 'describe', 'discuss', 'elaborate', 'derive']):
            return 'long_answer'
        elif any(kw in text_lower for kw in ['define', 'state', 'what is', 'name', 'list']):
            return 'short_answer'
        elif any(kw in text_lower for kw in ['calculate', 'solve', 'find', 'compute']):
            return 'numerical'
        else:
            return 'short_answer'
    
    @classmethod
    def extract_questions(cls, text: str) -> List[Question]:
        """
        Extract all questions from paper text.
        """
        questions = []
        current_section = None
        
        # Split into lines for processing
        lines = text.split('\n')
        current_question_lines = []
        current_number = None
        
        for line in lines:
            line = line.strip()
            if not line:
                continue
            
            # Check for section header
            for pattern in cls.SECTION_PATTERNS:
                match = re.search(pattern, line, re.IGNORECASE)
                if match:
                    current_section = f"Section {match.group(1)}"
                    break
            
            # Check for question start
            question_match = re.match(r'^\s*(\d+)[.)]\s+(.+)', line)
            q_match = re.match(r'^\s*[Qq]\.?\s*(\d+)[.)]?\s+(.+)', line)
            
            match = question_match or q_match
            
            if match:
                # Save previous question
                if current_number and current_question_lines:
                    full_text = ' '.join(current_question_lines)
                    clean_text, marks = cls.extract_marks(full_text)
                    q_type = cls.detect_question_type(clean_text)
                    
                    questions.append(Question(
                        number=current_number,
                        text=clean_text,
                        marks=marks,
                        section=current_section,
                        question_type=q_type
                    ))
                
                current_number = match.group(1)
                current_question_lines = [match.group(2)]
            elif current_number:
                # Continue current question
                current_question_lines.append(line)
        
        # Don't forget the last question
        if current_number and current_question_lines:
            full_text = ' '.join(current_question_lines)
            clean_text, marks = cls.extract_marks(full_text)
            q_type = cls.detect_question_type(clean_text)
            
            questions.append(Question(
                number=current_number,
                text=clean_text,
                marks=marks,
                section=current_section,
                question_type=q_type
            ))
        
        return questions


# Test
test_paper = """
PHYSICS EXAM
Total Marks: 50
Time: 2 Hours

Section A (Short Answer)

1. State Newton's first law of motion. (2 marks)

2. Define acceleration. Give its SI unit. (3 marks)

3. What is the difference between mass and weight? (2 marks)

Section B (Problems)

4. Calculate the force required to accelerate a 5 kg object at 10 m/s². (5 marks)

5. Derive the equation v² = u² + 2as from first principles. (8 marks)
"""

questions = QuestionExtractor.extract_questions(test_paper)
print(f"\nExtracted {len(questions)} questions:\n")
for q in questions:
    print(f"  Q{q.number} [{q.section}] ({q.marks}m, {q.question_type}): {q.text[:50]}...")

---
## 6. Complete Paper Parser

In [ ]:
class QuestionPaperParser:
    """
    Complete pipeline to parse question papers.
    """
    
    def __init__(self):
        self.metadata_extractor = MetadataExtractor
        self.question_extractor = QuestionExtractor
    
    def parse_file(self, file_path: str) -> QuestionPaper:
        """
        Parse a question paper file.
        """
        # Extract text
        raw_text = extract_text(file_path)
        return self.parse_text(raw_text)
    
    def parse_text(self, text: str) -> QuestionPaper:
        """
        Parse question paper from raw text.
        """
        # Extract metadata
        title = self.metadata_extractor.extract_title(text)
        subject = self.metadata_extractor.extract_subject(text)
        total_marks = self.metadata_extractor.extract_total_marks(text)
        time_limit = self.metadata_extractor.extract_time_limit(text)
        
        # Extract questions
        questions = self.question_extractor.extract_questions(text)
        
        # Extract sections
        sections = list(set(q.section for q in questions if q.section))
        
        # Calculate total marks if not found in text
        if total_marks == 0:
            total_marks = sum(q.marks for q in questions)
        
        # Extract instructions
        instructions = self._extract_instructions(text)
        
        return QuestionPaper(
            title=title,
            subject=subject,
            total_marks=total_marks,
            time_limit_minutes=time_limit,
            instructions=instructions,
            sections=sorted(sections),
            questions=questions,
            raw_text=text
        )
    
    def _extract_instructions(self, text: str) -> List[str]:
        """Extract instructions from paper."""
        instructions = []
        lines = text.split('\n')
        
        in_instructions = False
        for line in lines:
            line = line.strip()
            
            if 'instruction' in line.lower():
                in_instructions = True
                continue
            
            if in_instructions:
                # Stop at section or question start
                if re.match(r'^(Section|SECTION|\d+[.)])', line):
                    break
                if line:
                    instructions.append(line)
        
        return instructions[:5]  # Limit to 5 instructions
    
    def to_json(self, paper: QuestionPaper) -> str:
        """Export paper as JSON."""
        return json.dumps(paper.to_dict(), indent=2)


# Test complete pipeline
parser = QuestionPaperParser()
paper = parser.parse_text(test_paper)

print("="*60)
print("PARSED QUESTION PAPER")
print("="*60)
print(f"Title: {paper.title}")
print(f"Subject: {paper.subject}")
print(f"Total Marks: {paper.total_marks}")
print(f"Time: {paper.time_limit_minutes} minutes")
print(f"Sections: {paper.sections}")
print(f"Questions: {len(paper.questions)}")
print()
print("Questions:")
for q in paper.questions:
    print(f"  {q.number}. [{q.marks}m] {q.text[:60]}...")

---
## 7. Answer Key Association

In [ ]:
def add_answer_key(paper: QuestionPaper, answer_key: Dict[str, dict]) -> QuestionPaper:
    """
    Associate expected answers with questions.
    
    Args:
        paper: Parsed question paper
        answer_key: Dict mapping question number to {"answer": str, "keywords": list}
    
    Returns:
        Updated paper with answers
    """
    for question in paper.questions:
        if question.number in answer_key:
            key = answer_key[question.number]
            question.expected_answer = key.get("answer", "")
            question.keywords = key.get("keywords", [])
    
    return paper


# Example
answer_key = {
    "1": {
        "answer": "Newton's first law states that an object at rest stays at rest, and an object in motion stays in motion, unless acted upon by an external force.",
        "keywords": ["rest", "motion", "external force", "inertia"]
    },
    "2": {
        "answer": "Acceleration is the rate of change of velocity with respect to time. SI unit is m/s² (meters per second squared).",
        "keywords": ["rate of change", "velocity", "time", "m/s²"]
    },
    "4": {
        "answer": "Using F = ma, F = 5 × 10 = 50 N",
        "keywords": ["F = ma", "50", "N", "newton"]
    }
}

paper_with_answers = add_answer_key(paper, answer_key)

print("Questions with answers:")
for q in paper_with_answers.questions:
    if q.expected_answer:
        print(f"\nQ{q.number}: {q.text[:50]}...")
        print(f"   Answer: {q.expected_answer[:80]}...")
        print(f"   Keywords: {q.keywords}")

---
## 8. Export Module

In [ ]:
# Export JSON for integration
json_output = parser.to_json(paper_with_answers)
print("JSON Output:")
print(json_output)

In [ ]:
# Save module to utils
PAPER_PARSER_MODULE = '''
"""
Question Paper Parser

Extract questions, marks, sections from question papers.
"""
import re
import json
from typing import List, Dict, Optional, Tuple
from dataclasses import dataclass, field, asdict

@dataclass
class Question:
    number: str
    text: str
    marks: float
    section: Optional[str] = None
    question_type: str = "short_answer"
    expected_answer: Optional[str] = None
    keywords: List[str] = field(default_factory=list)

@dataclass
class QuestionPaper:
    title: str
    subject: str
    total_marks: float
    time_limit_minutes: int
    instructions: List[str] = field(default_factory=list)
    sections: List[str] = field(default_factory=list)
    questions: List[Question] = field(default_factory=list)
    
    def to_dict(self) -> dict:
        return {
            "title": self.title, "subject": self.subject,
            "total_marks": self.total_marks, "time_limit_minutes": self.time_limit_minutes,
            "sections": self.sections, "questions": [asdict(q) for q in self.questions]
        }

class QuestionPaperParser:
    MARK_PATTERNS = [r"\\((\\d+(?:\\.\\d+)?)\\s*(?:marks?|m)\\)", r"(\\d+)\\s*marks?$"]
    
    def parse_text(self, text: str) -> QuestionPaper:
        title = self._extract_title(text)
        subject = self._detect_subject(text)
        total_marks = self._extract_total_marks(text)
        time_limit = self._extract_time(text)
        questions = self._extract_questions(text)
        sections = list(set(q.section for q in questions if q.section))
        if total_marks == 0: total_marks = sum(q.marks for q in questions)
        return QuestionPaper(title, subject, total_marks, time_limit, [], sections, questions)
    
    def _extract_title(self, text): 
        for line in text.split("\\n")[:5]:
            if len(line.strip()) > 10: return line.strip()[:100]
        return "Question Paper"
    
    def _detect_subject(self, text):
        for subj in ["physics", "chemistry", "math", "biology", "english"]:
            if subj in text.lower(): return subj.capitalize()
        return "General"
    
    def _extract_total_marks(self, text):
        m = re.search(r"(?:total|max)\\s*marks?\\s*[:\\-]?\\s*(\\d+)", text.lower())
        return float(m.group(1)) if m else 0
    
    def _extract_time(self, text):
        m = re.search(r"time\\s*[:\\-]?\\s*(\\d+)\\s*(?:hours?|hrs?)", text.lower())
        return int(m.group(1)) * 60 if m else 0
    
    def _extract_questions(self, text) -> List[Question]:
        questions, section = [], None
        for line in text.split("\\n"):
            sec = re.search(r"Section\\s+([A-Z])", line, re.I)
            if sec: section = f"Section {sec.group(1)}"
            m = re.match(r"^\\s*(\\d+)[.)]\\s+(.+)", line)
            if m:
                txt, marks = self._extract_marks(m.group(2))
                questions.append(Question(m.group(1), txt, marks, section))
        return questions
    
    def _extract_marks(self, text):
        for p in self.MARK_PATTERNS:
            m = re.search(p, text, re.I)
            if m: return re.sub(p, "", text).strip(), float(m.group(1))
        return text.strip(), 0.0
'''

output_path = Path("../utils/paper_parser.py")
output_path.parent.mkdir(parents=True, exist_ok=True)
output_path.write_text(PAPER_PARSER_MODULE)
print(f"✅ Saved to {output_path.resolve()}")

---
## Summary

| Component | Purpose |
|-----------|--------|
| `extract_text()` | PDF/Image → Text |
| `MetadataExtractor` | Title, marks, time |
| `QuestionExtractor` | Questions, sections |
| `QuestionPaperParser` | Complete pipeline |
| `add_answer_key()` | Link expected answers |

**Output:** JSON with structured question paper data.